In [1]:
using Distributed

In [103]:
procs = addprocs(4) #choose carefully

4-element Array{Int64,1}:
  7
  8
  9
 10

In [104]:
length(procs)

4

In [105]:
@everywhere begin

const d = 4
const N = 640
const P = Int(floor(2*N/3)) #N
const nbits = max(53, Int(floor(P*log2(10))))

# choose a binary precision matching P as closely as we can.
setprecision(BigFloat, nbits)

const c_text = "0.5754"
const r_text = "0.8"
    
const c = BigFloat(c_text)
const r = BigFloat(r_text)

prefix = "feig_d$(d)_N$(N)_P$(P)_c$(c_text)_r$(r_text)_"
    
end

d, N, P, nbits, c, r

(4, 640, 426, 1415, 0.5754000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000005, 0.7999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999996)

In [106]:
@everywhere begin

import Base: <, <=, ==, >, >=
import Base: zero, one
import Base: abs
import Base: +, -, *, /, ^

struct Poly
    coeffs::Array{BigFloat, 1}
    cr::Tuple{BigFloat, BigFloat}
    Poly(a::Array{BigFloat, 1}, cr) = length(a) == 1+N ? new(a, cr) : error("wrong #coeffs")
end

abs(t::Poly) = sum(abs(a) for a in t.coeffs)
zero(p::Poly) = Poly(fill(BigFloat(0), N+1), p.cr)
one(p::Poly) = Poly([k==0 ? BigFloat(1) : BigFloat(0) for k in 0:N], p.cr)

(-)(p::Poly) = Poly(-p.coeffs, p.cr)

(*)(p::Poly, a::Number) = Poly(p.coeffs*a, p.cr)
(*)(a::Number, p::Poly) = p*a

(/)(p::Poly, a::BigFloat) = p*(1/a)

function +(p::Poly, a::BigFloat)
    coeffs = copy(p.coeffs)
    coeffs[1] += a
    Poly(coeffs, p.cr)
end

(+)(a::BigFloat, p::Poly) = p+a
(-)(p::Poly, a::BigFloat) = p+(-a)

(+)(p::Poly, q::Poly) = p.cr == q.cr ? Poly(p.coeffs+q.coeffs, p.cr) : error("!")
(-)(p::Poly, q::Poly) = p.cr == q.cr ? Poly(p.coeffs-q.coeffs, p.cr) : error("!")

(*)(a::Array{BigFloat, 1}, b::BigFloat) = [ak*b for ak in a]
(*)(b::BigFloat, a::Array{BigFloat, 1}) = a*b

(/)(a::Array{BigFloat, 1}, b::BigFloat) = [ak/b for ak in a]

function *(p::Poly, q::Poly)
    if p.cr != q.cr error("!") end
    coeffs = fill(BigFloat(0), N+1)
    for j in 0:N
        for k in 0:N
            if j+k <= N
                coeffs[1+j+k] += p.coeffs[1+j]*q.coeffs[1+k]
            end
        end
    end
    Poly(coeffs, p.cr)
end

basis_element(p::Poly, k::Integer) = Poly([j==k ? BigFloat(1) : BigFloat(0) for j in 0:N], p.cr)

function identity(p::Poly)
    c, r = p.cr
    coeffs = fill(BigFloat(0), N+1)
    coeffs[1] = c
    coeffs[2] = r
    Poly(coeffs, p.cr)
end

function diff(p::Poly)
    c, r = p.cr
    Poly([k == N+1 ? BigFloat(0) : k*p.coeffs[1+k] for k in 1:N+1], p.cr)/r
end

# Power
function russian(t, p::Integer)
    if p == 0
        return one(t)
    elseif p == 1
        return t
    end
    
    # ignore trailing zeros in binary expansion
    trailing = trailing_zeros(p) + 1
    p >>= trailing
    
    # repeatedly square
    while (trailing -= 1) > 0
        t *= t
    end
    
    # compute the rest
    u = t
    while p > 0
        trailing = trailing_zeros(p) + 1
        p >>= trailing
        while (trailing -= 1) >= 0
            t *= t
        end
        u *= t
    end
    u
end

(^)(t::Poly, p::Integer) = russian(t, p)

# Evaluation and composition of polynomial.
function (t::Poly)(x::Union{BigFloat, Poly, Complex{BigFloat}})
    c, r = t.cr
    u = (x-c)/r
    total = t.coeffs[N+1]*u + t.coeffs[N]
    for k in N-1:-1:1
        total = total*u + t.coeffs[k]
    end
    total
end

(t::Poly)(x::Number) = t(BigFloat(x))
    
end

In [107]:
@everywhere begin

coeffs = fill(BigFloat(0), N+1)
mu_infty = BigFloat("-1.5949013562288205644978287")
coeffs[1:2] = [BigFloat(1), mu_infty]
p0 = Poly(coeffs, (BigFloat(0), BigFloat(1)))

zero_Omega = zero(Poly(fill(BigFloat(0), N+1), (c, r)))
I_Omega = identity(zero_Omega)

g0 = p0(I_Omega)
    
end

In [108]:
@everywhere poly_basis = [basis_element(g0, k) for k in 0:N];

In [109]:
@everywhere begin

Q(z) = z^d
Qdash(z) = d == 2 ? 2*z : d*z^(d-1)

function T(f)
    x = identity(f)
    a = f(1)
    (1/a)*f(Q(f(x*Q(a))))
end

function T(g)
    x = identity(g)
    
    a = g(1)
    Qa = Q(a)
    xQa = x*Qa
    gxQa = g(xQa)
    QgxQa = Q(gxQa)
    gQgxQa = g(QgxQa)
    Tg = (1/a)*gQgxQa
end

function T_and_DT(g)
    x = identity(g)
    
    a = g(1)
    Qa = Q(a)
    xQa = x*Qa
    gxQa = g(xQa)
    QgxQa = Q(gxQa)
    gQgxQa = g(QgxQa)
    Tg = (1/a)*gQgxQa
    
    g_dash = diff(g)
    g_dashxQa = g_dash(xQa)
    g_dashQgxQa = g_dash(QgxQa)
    
    dgxQadm1 = d*gxQa^(d-1)
    
    # we should ensure that any subexpressions
    # that do not depend on delta_g
    # are computed in the outer function, here.
    
    function DTg(delta_g)
        delta_a = delta_g(1)
        delta_Qa = d*a^(d-1)*delta_a
        delta_xQa = x*delta_Qa
        delta_gxQa = delta_g(xQa) + g_dashxQa*delta_xQa
        delta_QgxQa = dgxQadm1*delta_gxQa
        delta_gQgxQa = delta_g(QgxQa) + g_dashQgxQa*delta_QgxQa
        delta_Tg = (-1/a^2)*delta_a*gQgxQa + (1/a)*delta_gQgxQa
    end
    
    Tg, DTg
end
    
end

In [110]:
@everywhere begin

function eye(T::Type, n::Integer)
    I::Array{T, 2} = [j==k ? T(1) : T(0) for k in 1:n, j in 1:n]
end

# be careful that matrix_elements is only used from 1 process.
# pmap can then be used below.
# for large degree, this should help significantly.
# when the function being applied is a closure,
# there might be a benefit to using CachingPool(workers())
# as an additional, second, argument to pmap.
# actually, here, DTp is not a closure...
    
function matrix_elements(L, basis)
    J_polys = pmap(L, basis)
    #J_polys = pmap(L, CachingPool(workers()), basis)
    J = [J_polys[k].coeffs[j] for j in 1:N+1, k in 1:N+1]
end
    
function newtonstep(p::Poly)
    Tp, DTp = T_and_DT(p)
    Fp = Tp - p
    J = matrix_elements(DTp, poly_basis)
    I = eye(BigFloat, N+1)
    q = Poly(p.coeffs - inv(J-I)*Fp.coeffs, p.cr)
end
    
function newton(p0::Poly, n::Integer)
    prev_error = BigFloat(10)^10
    for k in 1:n
        p1 = newtonstep(p0)
        error = abs(T(p1)-p1)
        if error < prev_error
            println("$k: $error")
            p0 = p1
            prev_error = error
        else
            break
        end
    end
    p0
end
    
end

## Skip if precomputed

In [10]:
ga = g0
gb = T(ga)
erra = abs(gb-ga)
for k in 1:20
    gc = T(gb)
    errb = abs(gc - gb)
    println("$(k) $(errb)")
    if errb < erra
        gb, ga = gc, gb
        erra = errb
    else
        break
    end
end

1 0.3006015216642470446521984467679237380294331712224122617124365340724579269807903375400942267126472896404757427907226256708478848465675727339376916724072188499989012904129525444868580525621087455567005341573956970409324664434241394115562717781016474787961858208739446850552127166240491675947610742969263604047233966977111869677603769242310460729185535362163066649163286314866882871078688290380508090689802675660474696872005558266
2 0.02346509242791335839763410389828405392921700389752231107669751601939221704775807066353953185988945456557759931222379703533384139259077949334787696037489095006284277864626999070731806643884309813530848135570313302356038748495155316914515540516870611649668845375365517749282444085381453447006561796099389982998271347203738963637189221842923097946134163334406319764819623953587273052276481612066563379991784389731849869316026687843
3 0.01804265742848504939541564980961978683248218871675699653646427197472191157810732070786613609603078974277517342550310776172754702815

In [ ]:
using Serialization
open(f -> serialize(f, ga), "$(prefix)ga.jls", "w")

In [11]:
ga = open(deserialize, "$(prefix)ga.jls")

Poly(BigFloat[-0.000151171351446015530128964258876835285301730175024148702051865988456910482717115526459734525633950239114624012482958269593847404801892686687975984265376797793809525940415064617858999836173124975217689230277838316608325402323635758163771694474921139644664669292951063190925119563242569005376690260425282498076603800465283737257892707390747569781295886872556422428718639780461002772075504701038429526599720976425170449146665223266, -1.256390384715622544898968398922438653789671105421868662128868476671052947549885242640091727465496749835249749510966528474882058496637408151574632220066926538808113647771425217544191808727041030542006154285550399596271506263866104518522694354770243063495626674551078398566289121690605306894301702684169760976913131509766462605385338747403418477061916530089882146315837344809165987487128827580860930926630935281230048173814872087, 0.24681564761436127195067059196266047446185750827753821985637876722673140392011023252740076256279108804781171145514992681

In [12]:
ga.coeffs[1]

-0.000151171351446015530128964258876835285301730175024148702051865988456910482717115526459734525633950239114624012482958269593847404801892686687975984265376797793809525940415064617858999836173124975217689230277838316608325402323635758163771694474921139644664669292951063190925119563242569005376690260425282498076603800465283737257892707390747569781295886872556422428718639780461002772075504701038429526599720976425170449146665223266

In [13]:
abs(T(ga)-ga)

5.625850989371967464511188368882404701607762628327613950554598714163346608384819096794454106601151922907423502537550398760274172203139833537340022130332401029752687342146751936694248679797981104609801798338853099832339481358909497996411706932328532919608296325683072671053549486462864264275212605908014949600195649606820281467333150055217338977111699605843600314727422303704497345800611751122379492547059920228831447996426502494e-10

In [14]:
#precompile
newton(g0, 0);

In [15]:
@time gn = newton(ga, 20);

1: 7.203578803580628163451869878961838321258491534392525143086072990094090627305089150698306494287345255097740235826438756261316429445657956538669542548594841805100410215977410776352574558713159790606779067214309461812091110882545322776239654404366788409612243719028099316676807369301151353166805178249611347374335002819243091379324069366985275743507393411987239141792220978760443706149595544390661741355899861391190097870993679553e-20
2: 2.941660694186793843972152208480775852616399307790543879865467929736136240119866728507136016783422193913710605494080565557347841576199987094708606899638759182963313548876893826643740342695337101913634528054612592791290128501857580963768035955147716594787637564989112331822518016470582956902078468084804700466303762042527787524508613781687177301861139910098027851215536311029635996115064782500305859897309128543289417152810958328e-39
3: 2.965065986915072238657615833742038152966386174041574340114172593597396879484201680152010795175058110417087876499383011305011

In [16]:
using Serialization
open(f -> serialize(f, gn), "$(prefix)g.jls", "w")

In [111]:
@everywhere begin
    
using Serialization

gn = open(deserialize, "$(prefix)g.jls")
    
end

In [18]:
gn(0)

1.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000035

In [19]:
gn(1)

-0.5916099166344381501396243543816289537902229891907558296390562608082701611002444465530968731159671843103521418006432697438637238931206828820779931596162409259411543052964276134709882939926870491577958887408376170145437404809085217681192114170711171042533082421009703580642260084834328708016478467785643980486155413892890080504401148407441724415402522204307760074169824796581420529101603145068347366296655913879718946454455618035

In [20]:
1/gn(1)

-1.690302971405244853343780150324161348228278059709561966682423263449739219088810551432766085786152919151931526308212594164105077561630908572940573192526278310204244018956025177655047935226236876644541321907107192676834935546971945672766866785148451453189013911194135568528212080475469696047558987391859329506662359225286618546743362667922598689747026129870126173400662013715316674245599135217770940296665254006891892598596168537

In [21]:
residue_approx = T(gn)-gn
abs(residue_approx)

9.984322956718816395337829831743471834860324862963760264754902451317442129874368294391788258792357511492940602139394764617345687230875053445051177871947186890187378019466840762902161844575987801421126678184077859622630407994728679100761423807621124123271951752930905747877842321665867009712353592384342318078808633776733731145178537498862291248083780927375470111124230872667382353502865351780203217852791032256409950692743118318e-426

In [22]:
ccc = gn.coeffs[1]

-0.0001511713350611291656840261159270919019237755709709817217414582918066104345259248288363569970343909936843052943870955651632007280456366801254175497134495694233204691022525525744921618127627868298306851128851750975628167740775703832655309932158811600523716590542438715231641835980685838847029840272029067002619921463929901470428506483651962794807390936479388709559585017492332329464494964582135859557037424246182049868511719220922

In [23]:
a = gn(1)
x = identity(gn)
xQa = x*Q(a)
psi(x) = (x-c)/r
norm1 = abs(psi(xQa))

0.7536423257237438864255931458452542313274839711478746703332428640580848482497972725610048619323420253628792246007404827541089029012226972704583855669734669365726443602365696824029560931963961815910622010125762551311722137227455570251344010755147125663155557116733018679516889677672574589801110471838232095959996509191841762098595729320268533988931249636724771412820484668564262259303586386217780369651092017830834808122900732035

In [24]:
QgxQa = Q(gn(xQa))
norm2 = abs(psi(QgxQa))

0.8034565068330793155746543122975066294353301987227608356292291707428176461485712142964345813680271194378271726019217026638596972383202399704472912561569990529424970690635105298344429025605102641948856423735041996324511653912126981904841508130792450028540581258475687621876840220373609799440396537371598978522630353043639988143284888051494670985705221818140002954687224479613493067043328534536946522628572219884461401049978589053

In [25]:
if norm1 >= 1 || norm2 >= 1
    println("Proof hopeless with these domains.")
end

In [26]:
Tgn, DTgn = T_and_DT(gn)

(Poly(BigFloat[-0.0001511713350611291656840261159270919019237755709709817217414582918066104345259248288363569970343909936843052943870955651632007280456366801254175497134495694233204691022525525744921618127627868298306851128851750975628167740775703832655309932158811600523716590542438715231641835980685838847029840272029067002619921463929901470428506483651962794807390936479388709559585017492332329464494964582135859557037424246182049868511719199743, -1.256390384839309113454966121731993593499508120265622773318149464633484336396804836636204709387546957569886985680440958954578313380665096451307611970486949817228483926263071247157541387050294375085102861980199101154532151910107399882664265223606888402126394556941200474897489421196716170925849460102196207429018722232057161323681395494982537300600697287478485375691945289183504373382813957410887741422672685470740426786680931147, 0.246815647559479889190375611990657185879440965219401197633844928315584601248688365610662659778189540687021413489761522

In [27]:
@time J = matrix_elements(DTgn, poly_basis);

12124.595848 seconds (31.13 M allocations: 1.402 GiB, 0.00% gc time)


In [28]:
using Serialization
open(f -> serialize(f, gn), "$(prefix)g.jls", "w")
open(f -> serialize(f, J), "$(prefix)jac.jls", "w")

## Continue...

In [11]:
@everywhere begin
    
using Serialization

gn = open(deserialize, "$(prefix)g.jls")
J = open(deserialize, "$(prefix)jac.jls")
    
end

In [30]:
gn.cr == (c, r)

true

In [12]:
Tgn, DTgn = T_and_DT(gn) # only if not done above

(Poly(BigFloat[-0.0001511713350611291656840261159270919019237755709709817217414582918066104345259248288363569970343909936843052943870955651632007280456366801254175497134495694233204691022525525744921618127627868298306851128851750975628167740775703832655309932158811600523716590542438715231641835980685838847029840272029067002619921463929901470428506483651962794807390936479388709559585017492332329464494964582135859557037424246182049868511719199743, -1.256390384839309113454966121731993593499508120265622773318149464633484336396804836636204709387546957569886985680440958954578313380665096451307611970486949817228483926263071247157541387050294375085102861980199101154532151910107399882664265223606888402126394556941200474897489421196716170925849460102196207429018722232057161323681395494982537300600697287478485375691945289183504373382813957410887741422672685470740426786680931147, 0.246815647559479889190375611990657185879440965219401197633844928315584601248688365610662659778189540687021413489761522

In [32]:
for (k, a) in enumerate(gn.coeffs)
    println("$(k-1): $(a)")
end

0: -0.0001511713350611291656840261159270919019237755709709817217414582918066104345259248288363569970343909936843052943870955651632007280456366801254175497134495694233204691022525525744921618127627868298306851128851750975628167740775703832655309932158811600523716590542438715231641835980685838847029840272029067002619921463929901470428506483651962794807390936479388709559585017492332329464494964582135859557037424246182049868511719220922
1: -1.256390384839309113454966121731993593499508120265622773318149464633484336396804836636204709387546957569886985680440958954578313380665096451307611970486949817228483926263071247157541387050294375085102861980199101154532151910107399882664265223606888402126394556941200474897489421196716170925849460102196207429018722232057161323681395494982537300600697287478485375691945289183504373382813957410887741422672685470740426786680931151
2: 0.24681564755947988919037561199065718587944096521940119763384492831558460124868836561066265977818954068702141348976152244967636

24: 1.004423856884389818876028509583228648350263819379420352448195165172144738732042416545584625187730712160024547219491300811120273864008745668836624029790480886853205245157679130766766506492721684656219331343887081523778748735304379567281603493705575050135194858613096602117358672816618669899115519055252334581588210686861019531232060153423648176892676592067056609802276440671313901464277147306988113842067567892812377816190096337e-12
25: 6.516615444194818766771801339398645021690529174550370054209442468552372471049663028493602266431928139766662324803584372655562350570399946057619530814949493628239963191665533637587779622232733757836175943576299713112423790877545218471101669005291492730958494592568909511323020171186034054303540973312463468343657414881892606287294852003870251292257964379758417383605203421048945238838534027796018241304096110771186798051949658764e-14
26: -1.07923762562418386337996634547191963882633014614196966353128071547620057388478898729783199855132887404536798348522748441

48: -3.870463730508515641009567390619034885474300280409532846777474245878762786333666355022129536665828209431871488561899636025018077506980061695921982335599418111562280991707436648650024589165798335082722291056757853189383506520081717651794292764419656100210126752039840205081966715065942142043449324889804635032492564068681493416170651459564654131674765567137459357967719167840977408683058904611141288444159121837319577709597697908e-25
49: 1.029952764194382096002914335138318074562181427043696962339027246888823435640941272367827715098608773709738652063356362851986430142450915285218937404398747710116655283641327406860928005081914053688714022141320180145087783426140037559609968512782760317156176366868730232425379345433173328189019917165877779480151270513116003077024300068924394991596110303097175913099656202684604652061354358332581617988951369182615057650755204589e-25
50: 9.05884236986270081917343292262101685724345261415155416027752642307827062144855181117060488650010780218061870330054656556

72: -8.788561059517135307141539090650667727675747761498916317057162627398641286470999253464884205563668127070755547573383273291969840525180206072401013567537070552365742030894409992538065754611238871653585178771852742357272313656044161946451403631125077219216626482400910592116678707110942254039945681720276023860551296674226149302867129657521012346935970394017981647011947943427217516001068227911492424792108473469444177229944541079e-39
73: -4.275993315397188803338834630863694143123300481132978928450540623510148807459192980589964780498479196428484395361799273060525578366405524562645840874656818724611075495592026811140952285392638657413022331445849142996484484650653451130199188875232808823981607561609186720493586984593019777853282193278041602502167005883355680247724525429681582812152787677760993150467566645721137700745274731652158856838860537428860575890081474216e-38
74: 1.1865108440081669778765795357395404822603613850055909221898165824341318428576901961250811116159141266535352823183428461

96: 5.408843658522156362438556118679492407044914056139380704185477693092955254393613891116554410151326817536419285190587716856045881211864606205219129618164269589005587460196277746714680227859613707092799838986886268834228821966719627184706517001047432312769036243485304485984253540168441698035486008623871787232987411057003283053377001508574266715810920861269254346507498440995906072623640171932113085342232179316260764912271754853e-50
97: -9.493951664959595631703556891774228197286607847311915601224461562941935817442617828847933915986405515093839048787620303362964107991464307885461794346841430884618270070795635016620959140152982670960107481057068151106187835041330365576447728058171424508222979395472049113728247797101756966284484444382477935525042491268460667849538227627293588525017057776194936092777287735665706738340060531109913024466946794731895601214283989212e-52
98: -4.7323210253777409387038676805909820389945472851781419499550112375192657283438932813604333013365778280076767488783138463

...

528: 9.410112103524858358502774014673696432990264374991462421809162406016752171209904502845025246259501954741104055918947585146273143027132064032950488334037512164231493244313710714423733269146192421800124339181464356587399575950633960920895557490715025419560909556995501938463004165395866576757439530237226961392120716560489511134355841975485723441342008351395984191787733231631504205514678534054556164019475407334535189590836245586e-274
529: -5.30356900323775840526972233843204989872383327671507964083843195948689303545859447501398351313931864637158834201066583814809179196129435118279525168177602631724979848241305229915148554286653971384728401863121603794172956410076460822524187460698482385924475471340081420467846720315960393103926068370473988766768265824549085904467694501030386000516454111964109235252049713748220584033249982212213004835422949968752787960253644039e-274
530: 5.0275588938740999988834837720484676647152165507244685716595461506646462731246218324729961939963583426548584309184315

552: 3.617160748012495267918394898625419501905556326612634357908092266840021503262070469612436371229105407880838014442945338904008114716733290384157611869033893311442966580004588806305151776490426277233868514357370906566937309086862797869506246759289429053137435927547731034171783296403766299633169719202866250287427993162791762405089470967078863258140067928633308329301406053719944034145788821554335801201759945063821562134812265121e-286
553: 1.112803698283531593781494263034127832370036009134329848461334162490929993499881180793895341850482072415241939711342393890106617051060742911682343051830428330212757408143862690282532030082154227533963544125406833782993284530779376687286403051360482942275986195641915960876738107892865344578941040826945110897879751498456733985655243181032054057547707860323833896319426219640048855408542331436625214939469940264387622521817974118e-286
554: -6.220543941229589191430030338333051591617578863673229528152772623218059547088803194721721964363188472091375884332884

576: -2.604033810399057766538204293014526211461244863566340690135112352724974767580740835688409903997649943944161387059321489588684821301462315819707926942665486222167873209834397672199613304122528571786243537273976905891679764902340290746228859309421225938807448796589603199133486973314782435737829574857381594562672756423773762970979533434230976888504445511860943593670462881296602110987070325809445993053735588214241900162317455591e-298
577: 4.220456709724586975585206767796028066177818953547442439457597807689385985943596076409976698755691705323429563427494603615024070323968394678853859737397626408119368815059253910794104614614185090864313143193798470437081487988251597248663788596203347415692679352752008538281295337671751319203887105577244779970185887118152794186546595100039610563947457430103728029476267908627100751651226701783806503540461158016230605571623060925e-299
578: 1.313473670320081457379617194913443163690042291067277270349877399572518509041905950432188810501693710257229299475053

600: 3.224339491847314468995512969740136689925805500312283976058839782448319701518668405948626873809627493917879185986080310509683075619666474556862972272467959152340928088165222781615014680349305943418718983765107860466778406615508971353437434671694353188683225051701227583293126471978814345145024091758080391493092341406470256151885805530864000094711838980802667871851417847904110396182670775012344133337764202786286759604832816675e-311
601: -3.056694091519540187353094769836134982818912544067837960561551589250023318380044483572814387030499213045339792208661189389210355910719889144323936393562002280328422184695088720880300698025744296976463175703657315066669835463866431951371612755724312230047483650885346900732417962033763635157441474820669416284739861037985572169171105357488668935429642370948141340361006619409819140299201301376613081636599133631691758882093075132e-311
602: 4.931198579967913390760891970408534248664434754853066836033218521161855982997153974820656651291687750403318024934670

624: 2.816835701388404005756368896905974952275347884687227497098962442785964671880158213721494206858796580736115336228586299430658135394756880905293842162835629893790581112648584599966379246511759219177417471415390619833096498381176725584652836908045604028459813348622031759695715404418088317403042019585432925019635499765316862140625754235050603175700906260236302339500918250550216024608863724682520141835398079232818633387059200667e-323
625: 3.804333257260647637448811392783060578398833683939336723184815203956328258088704204544265514059064620551355133100655417988570902520824416619042824704985628673325910252127656226216640363563026204425965732067176278969369818206319473594696999180188626149085793862026755295296510957579487090387284185542792193745981239326171012081997361511515890774952255319497153638971402336003474426665651093936439266948156250299616184035014113955e-324
626: -3.586421184524310536490093722349631429779404796577520123551837093021552313977605851318729140226787678745299525846761

In [112]:
a = gn(1)
alpha = 1/a

-1.690302971405244853343780150324161348228278059709561966682423263449739219088810551432766085786152919151931526308212594164105077561630908572940573192526278310204244018956025177655047935226236876644541321907107192676834935546971945672766866785148451453189013911194135568528212080475469696047558987391859329506662359225286618546743362667922598689747026129870126173400662013715316674245599135217770940296665254006891892598596168537

In [113]:
@everywhere begin

# interval datatype
struct Interval
    lo::BigFloat
    hi::BigFloat
    Interval(lo, hi) = lo > hi ? error("bad Interval") : new(lo, hi)
    Interval(a) = Interval(a, a)
    Interval(i::Interval) = new(i.lo, i.hi)
end

# membership
function in(a::Number, x::Interval)
    x.lo <= a <= x.hi
end

# subset
function in(x::Interval, y::Interval)
    y.lo <= x.lo && x.hi <= y.hi
end

(==)(x::Interval, y::Interval) = x.lo == y.lo && x.hi == y.hi

(<)(x::Interval, y::Interval) = x.hi < y.lo
(>)(x::Interval, y::Interval) = y<x

(<)(x::BigFloat, y::Interval) = x < y.lo
(<)(x::Interval, y::BigFloat) = x.hi < y

(>)(x::BigFloat, y::Interval) = y<x
(>)(x::Interval, y::BigFloat) = y<x

zero(i::Interval) = Interval(0)
one(i::Interval) = Interval(1)

function abs(x::Interval)
    if 0 in x
        return Interval(0, max(abs(x.lo), abs(x.hi)))
    else
        return Interval(min(abs(x.lo), abs(x.hi)), max(abs(x.lo), abs(x.hi)))
    end
end

import Base: setrounding

function +(x::Interval, y::Interval)
    setrounding(BigFloat, RoundDown)
    lo::BigFloat = x.lo+y.lo
    setrounding(BigFloat, RoundUp)
    hi::BigFloat = x.hi+y.hi
    setrounding(BigFloat, RoundNearest)
    Interval(lo, hi)
end

(-)(x::Interval) = Interval(-x.hi, -x.lo)
(-)(x::Interval, y::Interval) = x+(-y)

function +(x::Interval, a::Number)
    setrounding(BigFloat, RoundDown)
    lo::BigFloat = x.lo+a
    setrounding(BigFloat, RoundUp)
    hi::BigFloat = x.hi+a
    setrounding(BigFloat, RoundNearest)
    Interval(lo, hi)
end

(+)(a::Number, x::Interval) = x+a
(-)(x::Interval, a::Number) = x+(-a)
(-)(a::Number, x::Interval) = -x+a

function *(x::Interval, y::Interval)
    setrounding(BigFloat, RoundDown)
    lo::BigFloat = min(x.lo*y.lo, x.lo*y.hi, x.hi*y.lo, x.hi*y.hi)
    setrounding(BigFloat, RoundUp)
    hi::BigFloat = max(x.lo*y.lo, x.lo*y.hi, x.hi*y.lo, x.hi*y.hi)
    setrounding(BigFloat, RoundNearest)
    Interval(lo, hi)
end

function *(x::Interval, a::Number)
    if a >= 0
        setrounding(BigFloat, RoundDown)
        c::BigFloat = x.lo*a
        setrounding(BigFloat, RoundUp)
        d::BigFloat = x.hi*a
        setrounding(BigFloat, RoundNearest)
        Interval(c, d)
    else
        setrounding(BigFloat, RoundDown)
        e::BigFloat = x.hi*a
        setrounding(BigFloat, RoundUp)
        f::BigFloat = x.lo*a
        setrounding(BigFloat, RoundNearest)
        Interval(e, f)
    end
end

(*)(a::Number, x::Interval) = x*a

function iRec(x::Interval)
    if 0 in x
        error("division by zero")
    else
        setrounding(BigFloat, RoundDown)
        lo::BigFloat = 1/x.hi
        setrounding(BigFloat, RoundUp)
        hi::BigFloat = 1/x.lo
        setrounding(BigFloat, RoundNearest)
        Interval(lo, hi)
    end
end

(/)(x::Interval, y::Interval) = x*iRec(y)
(/)(x::Interval, a::Number) = x*(1/a)
(/)(a::Number, x::Interval) = a*iRec(x)

function ^(x::Interval, n::Integer)
    if n%2 == 1 || x.lo >= 0
        # odd power
        setrounding(BigFloat, RoundDown)
        a::BigFloat = x.lo^n
        setrounding(BigFloat, RoundUp)
        b::BigFloat = x.hi^n
        setrounding(BigFloat, RoundNearest)
        return Interval(a, b)
    elseif x.hi < 0
        setrounding(BigFloat, RoundDown)
        c::BigFloat = x.hi^n
        setrounding(BigFloat, RoundUp)
        d::BigFloat = x.lo^n
        setrounding(BigFloat, RoundNearest)
        return Interval(c, d)
    else
        setrounding(BigFloat, RoundUp)
        e::BigFloat = x.lo^n
        f::BigFloat = x.hi^n
        setrounding(BigFloat, RoundNearest)
        return Interval(0, max(e, f))
    end
end

function sqrt(x::Interval)
    half = BigFloat(1)/BigFloat(2)
    @assert half*BigFloat(2) == BigFloat(1)
    @assert x.lo >= BigFloat(0)
    setrounding(BigFloat, RoundDown)
    a = x.lo^half
    setrounding(BigFloat, RoundUp)
    b = x.hi^half
    setrounding(BigFloat, RoundNearest)
    Interval(a, b)
end

comparable(x::Interval, y::Interval) = x.hi <= y.lo || x.lo >= y.hi

setrounding(t::Type{Float64}, mode) = nothing

function bound_quasi_power_careful(k0, g_norm::Interval)
    @assert(g_norm.hi < Interval(1))
    @assert(g_norm.hi > Interval(0))
    # important to make the interval trivial, here
    g_norm = Interval(g_norm.hi, g_norm.hi)
    k = k0 + 1
    latest = k * g_norm ^ (k - 1)
    greatest_upper = latest.hi
    greatest_lower = latest.lo
    
    # a new upper bound strictly below a previous lower bound
    # indicates that we have passed the maximum.
    # at that point, the greatest upper bound found so far
    # and the greatest lower bound found so far
    # together bound the maximum of the graph.
    
    while latest.hi >= greatest_lower
        if latest.lo > greatest_lower
            greatest_lower = latest.lo
        end
        if latest.hi > greatest_upper
            greatest_upper = latest.hi
        end
        k = k + 1
        latest = k * g_norm ^ (k - 1)
        #println(latest)
    end
    Interval(greatest_lower, greatest_upper)
end

# here, we would make the argument that
# the rounded version of the quasi power
# is not order reversing at any point.

# this means that it should be sufficient to
# compute upper bounds only.

function bound_quasi_power(k0, g_norm::Interval)
    @assert(g_norm.hi < Interval(1))
    @assert(g_norm.hi > Interval(0))
    # important to make the interval trivial, here
    k = k0 + 1
    setrounding(BigFloat, RoundUp)
    latest = k * g_norm.hi ^ (k - 1)
    greatest_upper = latest
    while latest >= greatest_upper
        if latest > greatest_upper
            greatest_upper = latest
        end
        k = k + 1
        latest = k * g_norm.hi ^ (k - 1)
    end
    setrounding(BigFloat, RoundNearest)
    Interval(0, greatest_upper)
end

function balanced(i::Interval)
    a = max(abs(i.lo), abs(i.hi))
    Interval(-a, a)
end

function nonnegative(i::Interval)
    a = max(i.lo, BigFloat(0))
    @assert i.hi>=0
    Interval(a, i.hi)
end

function hull(v::Array{Interval, 1})
    lo = min([i.lo for i in v]...)
    hi = max([i.hi for i in v]...)
    Interval(lo, hi)
end

unit_vector(t::Type, k::Integer) = [j == k ? t(1) : t(0) for j in 0:N]
    
end

In [114]:
@everywhere begin

struct Rectangle
    re::Interval
    im::Interval
    Rectangle(re::Interval, im::Interval) = new(re, im)
    Rectangle(re, im) = new(Interval(re), Interval(im))
    Rectangle(re) = new(Interval(re), Interval(0))
end

in(x::Rectangle, z::Rectangle) = x.re in z.re && x.im in z.im
zero(z::Rectangle) = Rectangle(0, 0)
one(z::Rectangle) = Rectangle(1, 0)

(-)(z::Rectangle) = Rectangle(-z.re, -z.im)

(+)(z::Rectangle, w::Rectangle) = Rectangle(z.re+w.re, z.im+w.im)
(-)(z::Rectangle, w::Rectangle) = Rectangle(z.re-w.re, z.im-w.im)

(*)(z::Rectangle, x::Interval) = Rectangle(z.re*x, z.im*x)
(/)(z::Rectangle, x::Interval) = z*(1/x)

function reciprocal(z::Rectangle)
    d = z.re^2 + z.im^2
    Rectangle(z.re/d, -z.im/d)
end

(/)(z::Rectangle, w::Rectangle) = z*reciprocal(w)

(*)(z::Rectangle, w::Rectangle) = Rectangle(z.re*w.re-z.im*w.im, z.re*w.im+z.im*w.re)

(+)(z::Rectangle, x::Interval) = Rectangle(z.re+x, z.im)
(-)(z::Rectangle, x::Interval) = Rectangle(z.re-x, z.im)

# derived
(*)(x::Interval, z::Rectangle) = z*x
(/)(x::Interval, z::Rectangle) = Rectangle(x)/z
(+)(x::Interval, z::Rectangle) = z+x
(-)(x::Interval, z::Rectangle) = -z+x

# we don't have sqrt of an interval yet.
abs(z::Rectangle) = sqrt(z.re^2 + z.im^2)

(^)(z::Rectangle, p::Integer) = russian(z, p)

Rectangle(z::Complex{BigFloat}) = Rectangle(z.re, z.im)
    
end

In [115]:
@everywhere begin

struct Trunc
    coeffs::Array{Interval, 1}
    Trunc(a::Array{Interval, 1}) = length(a) == 1+N ? new(a) : error("wrong #coeffs")
end

abs(t::Trunc) = sum(abs(a) for a in t.coeffs)

(^)(t::Trunc, p::Integer) = russian(t, p)

zero(t::Trunc) = Trunc([Interval(0) for k in 0:N])
one(t::Trunc) = Trunc([k == 0 ? Interval(1) : Interval(0) for k in 0:N])

# Vector space essentials
(+)(t::Trunc, u::Trunc) = Trunc(t.coeffs + u.coeffs)
(-)(t::Trunc) = Trunc(-t.coeffs)
(*)(t::Trunc, a::Union{Number, Interval}) = Trunc(t.coeffs * a)

# Vector space derived
(-)(t::Trunc, u::Trunc) = t+(-u)
(/)(t::Trunc, a::Interval) = t*(1/a)
(*)(a::Union{Number, Interval}, t::Trunc) = t*a

# I think that (*)(Array{T, 1}, Number) is defined already by default.
# But since Interval is not a subtype of Number,
# we must define (*)(Array{Interval, 1}, Interval).
(*)(a::Array{Interval, 1}, b::Interval) = [ak*b for ak in a]
(*)(b::Interval, a::Array{Interval, 1}) = a*b

# Convenience
function +(t::Trunc, a::Interval)
    coeffs = copy(t.coeffs)
    coeffs[1] += a
    Trunc(coeffs)
end

function -(t::Trunc, a::Interval)
    coeffs = copy(t.coeffs)
    coeffs[1] -= a
    Trunc(coeffs)
end

(+)(a::Interval, t::Trunc) = t+a
(-)(a::Interval, t::Trunc) = (-t)+a

# Algebra
function *(t::Trunc, u::Trunc)
    coeffs = fill(Interval(0), N+1)
    for j in 0:N
        for k in 0:N-j
            coeffs[1+j+k] += t.coeffs[1+j]*u.coeffs[1+k]
        end
    end
    Trunc(coeffs)
end

# Evaluation and composition of polynomial. Compact, but could be made more efficient?
#(t::Trunc)(x::Union{Interval, Rectangle, Trunc}) = sum(a*x^(k-1) for (k, a) in enumerate(t.coeffs))
function (t::Trunc)(x::Union{Interval, Rectangle, Trunc})
    total = t.coeffs[N+1]*x + t.coeffs[N]
    for k in N-1:-1:1
        total = total*x + t.coeffs[k]
    end
    total
end

function bound_high_order_product(t::Trunc, u::Trunc)
    coeffs = Dict{Int64, Interval}()
    for j in 0:N
        for k in 0:N
            if j+k > N
                a::Interval = t.coeffs[1+j]*u.coeffs[1+k]
                if haskey(coeffs, j+k)
                    coeffs[j+k] += a
                else
                    coeffs[j+k] = a
                end
            end
        end
    end
    sum(abs(a) for (k, a) in coeffs)
end

abs_star(t::Trunc) = sum(abs(t.coeffs[1+k]) for k in 1:N)

diff(t::Trunc) = Trunc([k == N ? Interval(0) : (1+k)*t.coeffs[2+k] for k in 0:N])

end

In [116]:
@everywhere begin

struct StdBall
    P::Trunc
    H::Interval
    G::Interval
    
    # we enforce the intervals h, g to be of the form [0, b].
    StdBall(p::Trunc, h::Number, g::Number) = new(p, Interval(0, h), Interval(0, g))
    StdBall(p::Trunc, h::Interval, g::Interval) = new(p, Interval(0, h.hi), Interval(0, g.hi))
end

zero(b::StdBall) = StdBall(zero(b.P), Interval(0), Interval(0))
one(b::StdBall) = StdBall(one(b.P), Interval(0), Interval(0))

# Abs
abs(b::StdBall) = nonnegative(abs(b.P) + b.H + balanced(b.G))

# Vector space operations

# Negative
(-)(b::StdBall) = StdBall(-b.P, b.H, b.G)

# Scalar multiples
(*)(b::StdBall, i::Interval) = StdBall(b.P*i, b.H*abs(i), b.G*abs(i))
(/)(b::StdBall, i::Interval) = b*(1/i)

# Add and subtract
(+)(b::StdBall, c::StdBall) = StdBall(b.P+c.P, b.H+c.H, b.G+c.G)
(-)(b::StdBall, c::StdBall) = b+(-c)

# Constant add and subtract
(+)(b::StdBall, i::Interval) = StdBall(b.P+i, b.H, b.G)
(-)(b::StdBall, i::Interval) = StdBall(b.P-i, b.H, b.G)

# Derived
(*)(i::Interval, b::StdBall) = b*i
(+)(i::Interval, b::StdBall) = b+i
(-)(i::Interval, b::StdBall) = (-b)+i

function *(b::StdBall, c::StdBall)
    PP_P = b.P*c.P
    PP_H = bound_high_order_product(b.P, c.P)
    PH = abs(b.P)*c.H
    PG = abs(b.P)*c.G

    HP = b.H*abs(c.P)
    HH = b.H*c.H
    HG = b.H*c.G
    
    GP = b.G*abs(c.P)
    GH = b.G*c.H
    GG = b.G*c.G
    
    P = PP_P
    H = PP_H + PH + HP + HH + GH + HG
    G = PG + GP + GG
    
    StdBall(P, H, G)
end

(^)(t::StdBall, p::Integer) = russian(t, p)

function abs_star(g::StdBall)
    x = abs_star(g.P) + Interval(0, g.G.hi) + Interval(-g.G.hi, g.G.hi)
    Interval(max(0, x.lo), x.hi)
end

# function ball compose

function (f::StdBall)(g::StdBall)
    if ~(f.H == f.G == Interval(0)) && ~(abs(g) in Interval(0, 1))
        error("composition not defined")
    end
    ball_1 = f.P(g)
    ball_3 = StdBall(zero(f.P), Interval(0), f.G)
    g_norm = abs(g)^(N+1)
    g_star_norm = abs_star(g)^(N+1)
    ball_2 = StdBall(zero(f.P), f.H*g_star_norm, f.H*(g_norm - g_star_norm))
    ball_1 + ball_2 + ball_3
end

function (f::StdBall)(g) #could be g::AbstractRigorousScalar
    if ~(f.H == f.G == Interval(0)) && ~(abs(g) in Interval(0, 1))
        error("composition not defined")
    end
    g_norm = abs(g)^(N + 1)
    r = f.H * g_norm + f.G
    f.P(g) + Interval(-r.hi, r.hi)
end

# could absorb this into earlier definition
#(t::Trunc)(x::StdBall) = sum(a*x^(k-1) for (k, a) in enumerate(t.coeffs))
function (t::Trunc)(x::StdBall)
    total = t.coeffs[N+1]*x + t.coeffs[N]
    for k in N-1:-1:1
        total = total*x + t.coeffs[k]
    end
    total
end

function diff_compose(f::StdBall, g::StdBall)
    if (f.H.hi > 0 || f.G.hi > 0) && ~(abs(g) in Interval(0, 1))
        error("diff composition not defined")
    end
    ball_1 = diff(f.P)(g)
    g_norm = abs(g)
    if f.G == f.H == Interval(0)
        norm_sup_H = norm_sup_G = 0
    else
        norm_sup_H = bound_quasi_power(N, g_norm)
        norm_sup_G = bound_quasi_power(0, g_norm)
    end
    # some sacrifice is made here! absorbing into general term!
    # an alternative is to absord into both high-order term and
    # the final polynomial coefficient.
    ball_2 = StdBall(zero(f.P), Interval(0), f.H * norm_sup_H)
    ball_3 = StdBall(zero(f.P), Interval(0), f.G * norm_sup_G)
    ball_1 + ball_2 + ball_3
end

diff_compose(f::StdBall, t::Trunc) = diff_compose(f, StdBall(t, 0, 0))

function basis_element(b::StdBall, k::Integer)
    if k <= N
        return StdBall(Trunc(unit_vector(Interval, k)), 0, 0)
    else
        return StdBall(Trunc(fill(Interval(0), N+1)), 1, 0)
    end
end
    
end

In [117]:
@everywhere begin

struct Domain
    c::Interval
    r::Interval
    Domain(c::Interval, r::Interval) = r.lo <= 0 ? error("domain radius") : new(c, r)
end

in(z::Union{BigFloat, Interval}, d::Domain) = abs(z-d.c) < d.r
in(z::Rectangle, d::Domain) = abs(z-Rectangle(d.c, 0)) < d.r
(==)(d::Domain, e::Domain) = d.c==e.c && d.r==e.r
    
end

In [118]:
@everywhere begin

struct Ball
    std::StdBall
    dom::Domain
end

# negative
(-)(f::Ball) = Ball(-f.std, f.dom)

# scalar multiply
(*)(f::Ball, i::Interval) = Ball(f.std*i, f.dom)
(*)(i::Interval, f::Ball) = f*i
(/)(f::Ball, i::Interval) = f*(1/i)

(*)(f::Ball, a::Number) = f*Interval(a)
(*)(a::Number, f::Ball) = f*a

# ball add
(+)(f::Ball, g::Ball) = f.dom == g.dom ? Ball(f.std+g.std, f.dom) : error("domain mismatch")
(-)(f::Ball, g::Ball) = f.dom == g.dom ? Ball(f.std-g.std, f.dom) : error("domain mismatch")

# scalar add
(+)(f::Ball, a::Interval) = Ball(f.std+a, f.dom)
(-)(f::Ball, a::Interval) = Ball(f.std-a, f.dom)

# ball mul
(*)(f::Ball, g::Ball) = f.dom == g.dom ? Ball(f.std*g.std, f.dom) : error("domain mismatich")

# ball pow
(^)(f::Ball, n::Integer) = Ball(f.std^n, f.dom)

# ball eval/compose
(f::Ball)(i) = f.std((i - f.dom.c)/f.dom.r)
(f::Ball)(g::Ball) = Ball(f.std((g.std - f.dom.c)/f.dom.r), g.dom)

# diff_compose
diff_compose(f::Ball, g::Ball) = Ball(diff_compose(f.std, (g.std - f.dom.c)/f.dom.r)/f.dom.r, g.dom)

# allow differentiation for composition
function diff(f::Ball)
    df(g::Ball) = diff_compose(f, g)
end

# identity with respect to domain
function identity(f::Ball)
    coeffs = fill(Interval(0), N+1)
    coeffs[1] = f.dom.c
    coeffs[2] = f.dom.r
    Ball(StdBall(Trunc(coeffs), Interval(0), Interval(0)), f.dom)
end

basis_element(f::Ball, k::Integer) = Ball(basis_element(f.std, k), f.dom)

function Ball(p::Poly, h::Number, g::Number)
    c, r = p.cr
    dom = Domain(Interval(c), Interval(r))
    t = Trunc(map(Interval, p.coeffs))
    s = StdBall(t, h, g)
    b = Ball(s, dom)
end

abs(b::Ball) = abs(b.std)

function (m::Array{Interval, 2})(b::Ball, coeff_HH::Interval)
    PP = m*b.std.P.coeffs
    # assume all of G is in the polynomial part
    PG = [hull(m[1+k,:])*balanced(b.std.G) for k in 0:N]
    P = Trunc(PP + PG)
    H = (b.std.H + b.std.G)*abs(coeff_HH)
    G = Interval(0)
    Ball(StdBall(P, H, G), b.dom)
end
    
end

In [119]:
@everywhere dom = Domain(Interval(c), Interval(r))

In [120]:
@everywhere I = eye(BigFloat, N+1);

In [22]:
@everywhere Lambda_pp = map(Interval, inv(J-I))

In [23]:
@everywhere begin

function gauss(A_::Matrix)
    A = copy(A_)      
    m, n = size(A)
    @assert m <= n

    for k in 1:m
        
        # seek largest nonzero A[j,k] across j
        best, abs_best = k, abs(A[k,k])
        for i in k+1:m
            if abs(A[i,k]) > abs_best
                best, abs_best = i, abs(A[i,k])
            end
        end

        # swap rows to position pivot
        A[k,:], A[best,:] = A[best,:], A[k,:]

        # divide row by pivot
        pivot = A[k,k]
        for j in k:n
            A[k,j] /= pivot
        end

        # clear zeros above and below
        for k_that in 1:m
            if k_that != k
                ratio = A[k_that,k]/A[k,k]
                for j in k:n
                    A[k_that,j] -= ratio*A[k,j]
                end
            end
        end
    end
    A
end

#invert(A::Matrix) = gauss(hcat(A, eye(typeof(A[1,1]), size(A)[1])))[:,size(A)[2]+1:end]
    
end

In [24]:
# success here means that Lambda_pp, and hence Lambda itself,
# is invertible.
#@everywhere Lambda_pp_inv = invert(Lambda_pp);

In [25]:
@everywhere function Phi(b0::Ball, Lambda_pp::Array{Interval, 2})
    Tb0, DTb0 = T_and_DT(b0)
    Phib0 = b0 - Lambda_pp(Tb0-b0, Interval(1))
end

In [26]:
B0 = Ball(gn, 0, 0);

In [27]:
@time Phi_B0 = Phi(B0, Lambda_pp);

2598.555178 seconds (35.91 G allocations: 3.995 TiB, 19.87% gc time)


In [28]:
epsilon = abs(Phi_B0-B0)

Interval(0.0, 5.00350798828247084812451310396342471248688987534795120057327882377922968874700213747943820353653269472044073448685478379153919181603409224419910835869986881474406615414202831014282781722055551475017845679522666678527022117933015960297056644743902428175370051317727949369209559724074781543382157928635036191531026088993109778215042066253934424107531851658259809950749398901666780253433123404280614598496069098897879573682063695e-332)

In [29]:
rho_log10 = Int(round(0.5+log10(epsilon.hi)))
rho = BigFloat(10)^rho_log10

9.999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999992e-332

In [30]:
B1 = Ball(gn, 0, rho);

In [31]:
open(f -> serialize(f, B0), "$(prefix)b0.jls", "w")
open(f -> serialize(f, B1), "$(prefix)b1.jls", "w")

In [121]:
@everywhere begin
    
B0 = open(deserialize, "$(prefix)b0.jls")
B1 = open(deserialize, "$(prefix)b1.jls")
rho = B1.std.G.hi
    
end

In [122]:
@everywhere psi(x) = (x-B1.dom.c)/B1.dom.r
a = B1(Interval(1))

Interval(-0.5916099166344381501396243543816289537902229891907558296390562608082701611002444465530968731159671843103521418006432697438637238931206828820779931596162409259411543052964276134709882939926870491577958887408376170145437404809085217681192114170711171042533082421009703580642260084834328708016478467785643980486155413892890080504401149407441724415402522204307760074169824796581420529101603145068347366296655913879718946454455618082, -0.5916099166344381501396243543816289537902229891907558296390562608082701611002444465530968731159671843103521418006432697438637238931206828820779931596162409259411543052964276134709882939926870491577958887408376170145437404809085217681192114170711171042533082421009703580642260084834328708016478467785643980486155413892890080504401147407441724415402522204307760074169824796581420529101603145068347366296655913879718946454455618031)

In [34]:
@everywhere begin
T_B1, DT_B1 = T_and_DT(B1)
end

# do we really need this everywhere?
# can we serialise functions?

In [35]:
dB_H = basis_element(B1, N+1)
DPhi_B1_dB_H = -Lambda_pp(DT_B1(dB_H), Interval(1))
norm_H = abs(DPhi_B1_dB_H)

Interval(0.0, 1.869453152858791293404699582018340764227719265255326977269487440565734953837936964132768054369375955371458569466304924011555584042094714887997108357997754522006657238320435787998927421635039505817290176858209526448833638020448306891346855503766943377101528000511520641331523394542281272186170577670658454317981468019213107332438913298321433676863430556318086203916850923233807230699121591060725545006767703633792365932049723096e-59)

In [123]:
@everywhere E = [basis_element(B1, k) for k in 0:N+1];

In [37]:
# Precompute DT(B^1)(E_k) for all basis vectors, E_k.
@time DT_B1_E = pmap(DT_B1, E);

222979.309390 seconds (76.98 M allocations: 2.954 GiB, 0.00% gc time)


In [38]:
open(f -> serialize(f, DT_B1_E), "$(prefix)DT_B1_E.jls", "w")

In [39]:
@everywhere begin

DT_B1_E = open(deserialize, "$(prefix)DT_B1_E.jls")
    
function bound_kappa_naively(k::Int)
    DPhi_B1_E_k = E[k] - Lambda_pp(DT_B1_E[k]-E[k], Interval(1))
    norm = abs(DPhi_B1_E_k)
end

end

In [40]:
@time norms = pmap(bound_kappa_naively, [k for k in 1:N+2])

263.958690 seconds (1.17 M allocations: 58.897 MiB, 0.06% gc time)


642-element Array{Interval,1}:
 Interval(0.0, 2.219023058279401412627222942838289040758854445988893732110240393763239460155277205922159479510525897525286605884249135264970593058379710148469177990933684385242155084887045148976950235063383841789600859355860267239994342729095185298201081903084839350268447107433534564285831850293039648814954262122350600981669361296291056202046876636453441065770146484801678466687160550225135177714763129883847016499720348374546206798799038351e-326)
 Interval(0.0, 1.097876937645460354900745386732091632171747760217766805246198701617162750367817201659125497753631039095488685612301682226452478203729282081455161991427588171311003389087944047978408280233364238648342752059547438441622070597691129548222900853544030151770965408311574920410552452889840286193747254624709790518907485237694737171710925440847808874050178675168899306570398389707348668436325219090762008448176210664331768534524365079e-326)
 Interval(0.0, 7.6060848108063875934138204763474753788476843443224576

In [41]:
norms[end] = norm_H
kappa = hull(norms).hi

1.840245591277824025414002082557511364710238165618561220644833643589158012873498247112324613415157834047234691794383287964015564291711834715172736681229968821145633123866934912494357595746521132114974569369159038589685333820259554444932269851709740607748152601158271293299443472601284857624757029935573169609049087530950168650346408970085672898698754768335297513111262757818720264032873236743070162387017277927941156238579571485e-47

In [42]:
Phi_is_contractive = kappa < BigFloat(1)

true

In [43]:
fixed_point_exists = epsilon < rho*(Interval(1)-kappa)

true

In [44]:
println("Parameters:")
println("Power                     = $d")
println("Domain centre             = $c")
println("Domain radius             = $r")
println("Term G^u_0 (approx)       = $(gn.coeffs[1])")
println("Degree (reduced)          = $N")
println("Degree (original)         = $(N*d)")
println("Floating point type       = $(BigFloat)")
println("Precision (digits approx) = $P")
println("Precision (bits)          = $(precision(BigFloat))")
println()
println("Bounds:")
println("epsilon       = $(epsilon.hi)")
println("rho           = $rho")
#println("composition1  = $(norm1.hi)")
#println("composition2  = $(norm2.hi)")
println("kappa         = $kappa.hi")
println("rho*(1-kappa) = $((rho*(Interval(1)-kappa)).lo)")
println()
println("Conclusions:")
#println("proved domain extension   = $domain_extension")
println("proved phi is contractive = $Phi_is_contractive")
println("proved fixed point exists = $fixed_point_exists")
if fixed_point_exists
    println()
    println("Constants:")
    println("a = g(1)       = $(B1(1))")
    println("alpha = 1/g(1) = $(1/B1(1))")
end
println("Processors: 1+$(length(procs))")

Parameters:
Power                     = 4
Domain centre             = 0.5754000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000005
Domain radius             = 0.7999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999996
Term G^u_0 (approx)       = -0.00015117133

In [45]:
open(f -> serialize(f, [epsilon, rho, kappa]), "$(prefix)erk.jls", "w")

In [46]:
# now we must decide what to do with the high-order
# and general bounds we could take each general
# bound [0, g] and add [-g, g] to every polynomial
# term for the high-order bound [0, h] we form
# [-h, h]+[-g, g].  in this way we form what i call
# the augmented matrix $M$

M_aug = [Interval(0) for j in 1:N+2, k in 1:N+2]

for k in 1:N+2
    b_H = balanced(DT_B1_E[k].std.H)
    b_G = balanced(DT_B1_E[k].std.G)
    for j in 1:N+1
        M_aug[j,k] = DT_B1_E[k].std.P.coeffs[j] + b_G
    end
    M_aug[N+2,k] = b_H + b_G
end
M_aug;

# ignoring the high-order parts yields the so-called PP matrix
M_PP = M_aug[1:N+1,1:N+1];

In [47]:
# we now find approximate eigvenvalues and vectors of the PP matrix

In [48]:
using GenericSchur

M_approx = map(i -> i.lo, M_PP)
@time vals_rev, vecs_rev = GenericSchur.eigen(M_approx);

1655.596536 seconds (14.12 G allocations: 1.612 TiB, 16.71% gc time)


In [49]:
vals = reverse(vals_rev)

641-element Array{Complex{BigFloat},1}:
      8.163158323607492932358696175387889811410458812092228420930400167230387207541840632011988411850526712859224599717030287533801686604200987376389684699029288086345132922319405168846722965781922245711136080900743606402153576035706139584071950268675597868866336504785817704769910299591424972706160429402735730013896684663010166203605177954152853688485952860485230723016909515558680306702544767959868925568385049547658749962815705989 + 0.0im
      7.284686217073343364308930567995553069478046619799790659072121290188346214350676200657264503136037114707843578669255573693322112159491701670560272610414283470959822878732902387885867206416656818950731011658106317312791658163233662677746542527784419483203624379024983698686814670240496631580597051641021952709316631727445889929394505903908077421261831707738467733534153919932776458682182953949064141383807688711358719863272692374 + 0.0im
     0.2918384089835451045309969614572281815335153719467096530012880164024

In [50]:
open(f -> serialize(f, vals), "$(prefix)val.jls", "w")

In [51]:
vecs_rev_re = map(x->BigFloat(x), vecs_rev)
vecs = vecs_rev_re[1:N+1,N+1:-1:1];

In [52]:
normalised_T = [vecs[:,k]/sum(map(abs, vecs[:,k])) for k in 1:N+1]
vecs = [normalised_T[k][j] for j in 1:N+1, k in 1:N+1];

In [53]:
eigen_error = sum(map(abs, M_PP*vecs[:,1] - vals[1]*vecs[:,1]))

Interval(0.0, 2.349577835658715281479233495206974207614620286656574760103551880961423004799783681501941551245247951414945307537133915860803892557497033161316593532643428629487941966371994710280276639902410318212616682697543599167575827205391875250864399719196452093494918329658145038089338896400462059287460066278558454735689594750901769854109192553307429414439106511551954052495467165163245237438600802724135106885197283429491917610808781624e-326)

In [124]:
@everywhere begin

span(i::Interval) = Interval(0, (i-i).hi)

phi(p::Poly) = p.coeffs[1]
phi(b::Ball) = b.std.P.coeffs[1] + balanced(b.std.G)
    
end

In [55]:
phi(B0)

Interval(-0.0001511713350611291656840261159270919019237755709709817217414582918066104345259248288363569970343909936843052943870955651632007280456366801254175497134495694233204691022525525744921618127627868298306851128851750975628167740775703832655309932158811600523716590542438715231641835980685838847029840272029067002619921463929901470428506483651962794807390936479388709559585017492332329464494964582135859557037424246182049868511719220922, -0.0001511713350611291656840261159270919019237755709709817217414582918066104345259248288363569970343909936843052943870955651632007280456366801254175497134495694233204691022525525744921618127627868298306851128851750975628167740775703832655309932158811600523716590542438715231641835980685838847029840272029067002619921463929901470428506483651962794807390936479388709559585017492332329464494964582135859557037424246182049868511719220922)

In [56]:
span(phi(B0))

Interval(0.0, 0.0)

In [57]:
phi(B1)

Interval(-0.0001511713350611291656840261159270919019237755709709817217414582918066104345259248288363569970343909936843052943870955651632007280456366801254175497134495694233204691022525525744921618127627868298306851128851750975628167740775703832655309932158811600523716590542438715231641835980685838847029840272029067002619921463929901470428507483651962794807390936479388709559585017492332329464494964582135859557037424246182049868511719220924, -0.000151171335061129165684026115927091901923775570970981721741458291806610434525924828836356997034390993684305294387095565163200728045636680125417549713449569423320469102252552574492161812762786829830685112885175097562816774077570383265530993215881160052371659054243871523164183598068583884702984027202906700261992146392990147042850548365196279480739093647938870955958501749233232946449496458213585955703742424618204986851171922092)

In [58]:
span(phi(B1))

Interval(0.0, 2.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000003990116063274391563189005228863454063761389329488622199659186141579680487805077898499097794365550173911370857105429998298716815504060981435389963118927861810219212768663451181011828755816888701345452361315458401534805767641592384564183248302392093164467232917283491610388797387957540926743445533441233175399274398695821227157731e-331)

In [59]:
i = 2
val_approx = vals[i]
vec_approx = vecs[1:end,i]

# normalize properly
f0_poly_coeffs = vec_approx/vec_approx[1]*val_approx.re
f0_poly_coeffs[1]

7.284686217073343364308930567995553069478046619799790659072121290188346214350676200657264503136037114707843578669255573693322112159491701670560272610414283470959822878732902387885867206416656818950731011658106317312791658163233662677746542527784419483203624379024983698686814670240496631580597051641021952709316631727445889929394505903908077421261831707738467733534153919932776458682182953949064141383807688711358719863272692374

In [60]:
f0_poly = Poly(f0_poly_coeffs, (c, r))

Poly(BigFloat[7.284686217073343364308930567995553069478046619799790659072121290188346214350676200657264503136037114707843578669255573693322112159491701670560272610414283470959822878732902387885867206416656818950731011658106317312791658163233662677746542527784419483203624379024983698686814670240496631580597051641021952709316631727445889929394505903908077421261831707738467733534153919932776458682182953949064141383807688711358719863272692374, 7.166839843174802185240988106002383404418656314690186789648785834704922987576719243423747105833600868983528368822807355106744968503966842727172637476672561126065250847979036977265282163648424498490228557339085255795987567947340782906010978017482737508399200239326787156113815676261978476894320642576943019633691156104105079425096861857307660331421644870167263516375907846185639480336332144001433732588455779528120136035759004029, -5.433001676297059973591003061189907297321764946423042071989684167131989436694223409175519688600428313560673766494510982519528

In [61]:
abs(DTgn(f0_poly)-f0_poly*vals[i].re)

1.94724799028777981048327441959076576168990150234999882847239214092333867474139157063898633204298380215522885176356132247277649546632634201097957758109263854801603218202320323053155572158380272402717726989706878933992858200212306043325040497750689645053610310815630049127173657422914204184873121930130331707489904333498758103705284995090564425811194724424614320873405746211282303124263600675308046282446864077595215366404908859e-325

In [62]:
@everywhere begin
    
function newtonstep2(p::Poly)
    Fp = DTgn(p) - p.coeffs[1]*p
    Z = [BigFloat(0) for j in 1:N+1,k in 1:N+1]
    for j in 1:N+1
        for k in 1:N+1
            if k == 1
                Z[j, k] += p.coeffs[j]
            end
            if j == k
                Z[j, k] += p.coeffs[1]
            end
        end
    end
    q = Poly(p.coeffs - inv(J-Z)*Fp.coeffs, p.cr)
    q, abs(Fp)
end

function newton2(p0::Poly, n::Integer)
    prev_error = BigFloat(10)^10
    for k in 1:n
        p1, error = newtonstep2(p0)
        if error < prev_error
            println("$k: $error")
            p0 = p1
            prev_error = error
        else
            break
        end
    end
    p0
end
    
end

In [63]:
@time fn = newton2(f0_poly, 20);

1: 1.94724799028777981048327441959076576168990150234999882847239214092333867474139157063898633204298380215522885176356132247277649546632634201097957758109263854801603218202320323053155572158380272402717726989706878933992858200212306043325040497750689645053610310815630049127173657422914204184873121930130331707489904333498758103705284995090564425811194724424614320873405746211282303124263600675308046282446864077595215366404908859e-325
2: 3.014718777125064465890081956000256277860682409569652086161501539220594149300880260980254848327040006715065229143357797071919483132643463508396212275320949050628944499296744117320157083356942186830325818988918881686431799076193431252998436801436239514182342141536657598362538488974622816796111883230213641933895925275241347935118224118893932439928411909866219049789740792319015075694786163390653389672412124525082767428340335706e-424
3: 2.86434208577781687028656222414119506030592512855889670422300142868077172828989536035618201499601748580412210379148731506259

In [65]:
workers()

4-element Array{Int64,1}:
 2
 3
 4
 6

In [66]:
@time Delta_PP = matrix_elements(DTgn, poly_basis);

12335.822781 seconds (31.27 M allocations: 1.407 GiB, 0.00% gc time)


In [67]:
Delta_mod = [BigFloat(0) for j in 1:N+1,k in 1:N+1]
for j in 1:N+1
    for k in 1:N+1
        Delta_mod[j,k] = Delta_PP[j,k]
        if k == 1
            Delta_mod[j,k] = Delta_mod[j,k] - fn.coeffs[j]
        end
        if j == k
            Delta_mod[j,k] = Delta_mod[j,k] - fn.coeffs[1]
        end
    end
end
Delta_mod;

In [68]:
L_pp = map(Interval, inv(Delta_mod)); #invert

# now we check that the resulting interval matrix is invertible
#@time invert(L_pp);

In [69]:
V0 = Ball(StdBall(Trunc(map(Interval, fn.coeffs)), BigFloat(0), BigFloat(0)), dom);

In [70]:
eigenfunction_error = abs(V0*phi(V0) - DT_B1(V0))

Interval(0.0, 3.797730411468940253319363694563512066566372619185047054599605286347465406893652589116766406637857677386860234125988216543922522756477211964965980644967733991806303104991170253148960157856538895942211970578763268398863968472224543995748808490031729044687099031587318892179109544307820031842647814159591041338905545242344521539300112622142847843129178387097439325165015340586347067285624103970348883994544768851544606228452516756e-327)

In [71]:
V0.dom == B1.dom

true

In [72]:
open(f -> serialize(f, L_pp), "$(prefix)L_pp.jls", "w")
open(f -> serialize(f, V0), "$(prefix)V0.jls", "w")

In [73]:
@everywhere begin

L_pp = open(deserialize, "$(prefix)L_pp.jls")
V0 = open(deserialize, "$(prefix)V0.jls")
    
function Psi(v::Ball, L_pp::Array{Interval, 2})
    Psi_v = v - L_pp(DT_B1(v)-v*phi(v), -Interval(1)/V0.std.P.coeffs[1])
end
    
end

In [74]:
@time epsilon_hat = abs(Psi(V0, L_pp)-V0)

1293.195021 seconds (17.98 G allocations: 2.000 TiB, 20.01% gc time)


Interval(0.0, 2.899840037888097505371202931756303401807189971427873561302841238510739281403559251618152844234124015134972640589874103674097997980767000115125547480524875256456181865945304757341111077193846486154458253294132850985940209377927886770534930537549391605962514697446770548707222741181404666647767969149895899245843444883602288890448092135298425146321188711791568434379529642408547489442616606768726483843252116158778217120860336784e-326)

In [75]:
rho_hat = BigFloat(10)^Int(round(0.5+log10(epsilon_hat.hi)))

1.0e-325

In [76]:
V1 = Ball(fn, 0, rho_hat);

In [77]:
open(f -> serialize(f, V1), "$(prefix)V1.jls", "w")

In [78]:
println(phi(V1).lo)
println(phi(V1).hi)
println(1+rho_hat)

7.284686217073343364308930567995553069478046619799790659072121290188346214350676200657264503136037114707843578669255573693322112159491701670560272610414283470959822878732902387885867206416656818950731011658106317312791658163233662677746542527784419483203624379024983698686814670240496631580597051641021952709316631727445889929294524519109263637047505274079110661603271448446546152013276912994803976477615680877795781485616671252
7.284686217073343364308930567995553069478046619799790659072121290188346214350676200657264503136037114707843578669255573693322112159491701670560272610414283470959822878732902387885867206416656818950731011658106317312791658163233662677746542527784419483203624379024983698686814670240496631580597051641021952709316631727445889929494524519109263637047505274079110661603271448446546152013276912994803976477615680877795781485616671259
1.00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [79]:
rho_hat

1.0e-325

In [80]:
@everywhere begin
    
V1 = open(deserialize, "$(prefix)V1.jls")

function DPsi(b::Ball, L_pp::Array{Interval, 2})
    function DPsi_b(db::Ball)
        DF = DTb1(db)-phi(b)*db-phi(db)*b
        DPsi_b_db = db - L_pp(DF, -Interval(1)/phi(V0))
    end
    DPsi_b
end

function bound_kappa_hat_naively(k::Int)
    b = V1
    DPsi_b_es_k = E[k] - L_pp(DT_B1_E[k]-phi(b)*E[k]-phi(E[k])*b, -Interval(1)/phi(V0))
    norm = abs(DPsi_b_es_k)
end
    
end

In [81]:
EH = basis_element(V1, N+1)
DT_G_EH = DT_B1(EH)
DPsi_V1_EH = (Interval(1)-phi(V1)/phi(V0))*EH -L_pp(DT_G_EH, -Interval(1)/phi(V0))
norm_H = abs(DPsi_V1_EH)

Interval(0.0, 2.607153108399861722676500474362528755011660517434032535598367656748578228318005317185888720804948268988428150695548672849612715989021110056158782485581989410239481009945463638883489001690691570614375219059128132452719841723063240849590542101624493449897746297582473798585492148117724113911097635960355050878312473316090162948748624262129988773653765966938277109688935637877341152992924116727876946878150706749408189707855719317e-60)

In [82]:
@time norms_v = pmap(bound_kappa_hat_naively, [k for k in 1:N+2])

278.224520 seconds (330.88 k allocations: 15.251 MiB)


642-element Array{Interval,1}:
 Interval(0.0, 9.155350309139510873154355394761657327918333852603480100715625171775936371387014678860245460435536204364146038644168182618926116287209195769646630754457947350439853749069794529144893162972354218277320418837152740118540099969582252967268061932272899558841173638028898186866320442781018776359695357028567077947237746307439941805176845084557482669930420653845955002938012641243386986474196787740701138836196809049464750399179598135e-324)
 Interval(0.0, 1.391515307897778987786879291698949047212522046324961289064614755283628647807019492774449881408931902185125370285899108446898015758489942255599975767954835033240928110324034371573714978053244802914385385636281075409360832291027776202374950362354017702182325501214683204741942515500041747830478308813022062064772853639014832953919341331138189578360906508177741018847607592723701008265604171398351760590855945993689730416638124536e-325)
 Interval(0.0, 1.0086765845649183015591390673938965341466046016344161

In [83]:
norms_v[end] = norm_H
kappa_hat = hull(norms_v)

Interval(0.0, 2.526183745519173875403878539863771905787784300528043917761365887102743165659481747905603668194374938126195919848007206062805871462206795652898227119160703336610418915341819020971918378970270926157470864222049983368543181707032249453581589285497158318332080793257492830916573082282807798663528657516831030868007208404616291763377932670887260607181120334472031890597880716442334302003161924495388175439150101880303575777470242446e-48)

In [84]:
epsilon_hat < rho_hat*(Interval(1)-kappa_hat)

true

In [85]:
epsilon_hat

Interval(0.0, 2.899840037888097505371202931756303401807189971427873561302841238510739281403559251618152844234124015134972640589874103674097997980767000115125547480524875256456181865945304757341111077193846486154458253294132850985940209377927886770534930537549391605962514697446770548707222741181404666647767969149895899245843444883602288890448092135298425146321188711791568434379529642408547489442616606768726483843252116158778217120860336784e-326)

In [86]:
rho_hat*(Interval(1)-kappa_hat)

Interval(9.99999999999999999999999999999999999999999999997473816254480826124596121460136228094212215699471956082238634112897256834340518252094396331805625061873804080151992793937194128537793204347101772880839296663389581084658180979028081621029729073842529135777950016631456818292967750546418410714502841681667919206742507169083426917717192201336471342483168969131992791595383708236622067329112739392818879665527968109402119283557665696e-326, 1.0e-325)

In [87]:
println(phi(V1).lo)
println(phi(V1).hi)
println(1+rho_hat)

7.284686217073343364308930567995553069478046619799790659072121290188346214350676200657264503136037114707843578669255573693322112159491701670560272610414283470959822878732902387885867206416656818950731011658106317312791658163233662677746542527784419483203624379024983698686814670240496631580597051641021952709316631727445889929294524519109263637047505274079110661603271448446546152013276912994803976477615680877795781485616671252
7.284686217073343364308930567995553069478046619799790659072121290188346214350676200657264503136037114707843578669255573693322112159491701670560272610414283470959822878732902387885867206416656818950731011658106317312791658163233662677746542527784419483203624379024983698686814670240496631580597051641021952709316631727445889929494524519109263637047505274079110661603271448446546152013276912994803976477615680877795781485616671259
1.00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

# Critical scaling of additive noise

In [125]:
@everywhere function make_noise_functions(G)
    
    a = G(1)
    
    X = identity(G)
    XQa = X*Q(a)
    GXQa = G(XQa)
    QGXQa = Q(GXQa)
    GQGXQa = G(QGXQa)
    
    Gdash = diff(G)
    GdashQa = Gdash(XQa)
    GdashQGXQa = Gdash(QGXQa)
    
    t2 = (1/a^2)
    t3 = ((1/a)*GdashQGXQa*Qdash(GXQa))^2
    
    function T_noise(H)
        term2 = t2*H(QGXQa)
        term3 = t3*H(XQa)
        lhs = term2 + term3
        return lhs
    end

    function DT_noise(H)
        function DT_noise_H(K)
            term2 = t2*K(QGXQa)
            term3 = t3*K(XQa)
            lhs = term2 + term3
            return lhs
        end
        return DT_noise_H
    end

    function F_noise(H)
        lhs = T_noise(H)
        rhs = phi(H)^2*H
        return lhs - rhs
    end
    
    function DF_noise(H)
        function DF_noise_H(K)
            term2 = t2*K(QGXQa)
            term3 = t3*K(XQa)
            lhs = term2 + term3
            rhs = 2*phi(H)*phi(K)*H + phi(H)^2*K
            return lhs - rhs
        end
        return DF_noise_H
    end
    
    return T_noise, DT_noise, F_noise, DF_noise
end

In [89]:
T_noise, DT_noise, F_noise, DF_noise = make_noise_functions(gn)

(T_noise, DT_noise, F_noise, DF_noise)

In [126]:
function newtonstep3(p::Poly)
    F_noise_p = F_noise(p)
    
    DF_noise_p = DF_noise(p)
    Gamma_noise_inv = matrix_elements(DF_noise_p, poly_basis);
    
    q = Poly(p.coeffs - inv(Gamma_noise_inv)*F_noise_p.coeffs, p.cr)
    q, abs(F_noise_p)
end

function newton3(p0::Poly, n::Integer)
    basis = [basis_element(p0, k) for k in 0:N]
    prev_error = BigFloat(10)^10
    for k in 1:n
        p1, error = newtonstep3(p0)
        if error < prev_error
            println("$k: $error")
            p0 = p1
            prev_error = error
        else
            break
        end
    end
    p0
end

newton3 (generic function with 1 method)

In [127]:
v_noise_coeffs = BigFloat[8.2439108542525868183984625308166, -6.0314979598335030543896628404458, -0.99906557440532477623573316007567, 2.6365397214358035500680010442941, -0.81393255099831777701118943916041, -0.2916501997294583937683387447899, 0.27455204544220932923048986000544, -0.04629827836891161318089224352731, -0.03003398317265670879893203510508, 0.017474018792551148952490914683149]

10-element Array{BigFloat,1}:
  8.243910854252586517532108700834214687347412109375         
 -6.0314979598335032306977154803462326526641845703125        
 -0.9990655744053247300229259053594432771205902099609375     
  2.63653972143580350717684268602170050144195556640625       
 -0.8139325509983177386885699888807721436023712158203125     
 -0.29165019972945838144795516200247220695018768310546875    
  0.27455204544220934703702141632675193250179290771484375    
 -0.046298278368911614799596776492762728594243526458740234375
 -0.03003398317265670858233761464362032711505889892578125    
  0.017474018792551150591751962792841368354856967926025390625

In [128]:
v_noise_0 = Poly([k <= length(v_noise_coeffs) ? v_noise_coeffs[k] : BigFloat(0) for k in 1:N+1], (c, r))
v_noise_0(0), phi(v_noise_0)

(10.962100567341411600356356227757321671288435758151442727745673681727378578898424166254699230194091796875, 8.243910854252586517532108700834214687347412109375)

## Clear some space?

In [129]:
# aim to clear some variables
@everywhere begin
    DT_B1_E = 0
end

0

In [130]:
v_noise_n = newton3(v_noise_0, 20);

1: 0.45483785239563070905539231617001742436537083809062642544237848498642139288810159015770201347139497624853688810096647502735424848704957414964839765504452856807742088525454132640706109355789668970643955758462115644703513517872927099142821400179562303686574109601172464854393807831371613461490258703384830550099638296691849924124655364245804474530186706373416348058380616167237295900738389515798976030678992722453788572627245969
2: 3.303755163016733043677107040746734383106997034128501076089482965826972058036696441889777900172364316809021654086190249076971593489783247217247654529691888319281782190043926816829098306961283850207879885631507517599893990225984586592481613458543879361289360295451102948344242508180204322877893871871772410999626649305971181147178638875181732242494813696411307262473172277611490250778804644942640782972197005889108471633320295843e-17
3: 2.47464309052560811945442001253620403794582969455250517501272320019516790910696459983913945253463871063253789409375715993564804624

In [132]:
open(f -> serialize(f, v_noise_n), "$(prefix)noise_n.jls", "w")

@everywhere v_noise_n = deserialize("$(prefix)noise_n.jls")

In [133]:
phi(v_noise_n)

8.243910854252586818398462365029237616067317766624058409262192568256536639241425626899642047207578424223008736898322349635107173282537439471191666888923240182781145434355705947708003779852383166834676598572907048759876424584766481825677074055956898429768493270881184491967881214627576709080151177052580323304160627899933502176851169469283992307076121371557451839412289065433857567808741456796795611457473020484059192082775745046

In [134]:
@time Lambda_PP_inv_noise = matrix_elements(DF_noise(v_noise_n), poly_basis);

9885.618899 seconds (22.96 M allocations: 1.080 GiB, 0.00% gc time)


In [135]:
@time Lambda_PP_noise = map(Interval, inv(Lambda_PP_inv_noise));

161.441642 seconds (1.40 G allocations: 167.378 GiB, 14.07% gc time)


In [136]:
open(f -> serialize(f, Lambda_PP_noise), "$(prefix)Lambda_PP_noise.jls", "w")

@everywhere begin
Lambda_PP_noise = deserialize("$(prefix)Lambda_PP_noise.jls");
end

In [137]:
#@time Lambda_PP_noise_inv_check = inv(Lambda_PP_noise);

In [138]:
@everywhere begin
    
T_noise, DT_noise, F_noise, DF_noise = make_noise_functions(B1)

function Phi_noise(b::Ball)
    b - Lambda_PP_noise(F_noise(b), -1/phi(b)^2)
end
    
end

In [139]:
B0_noise = Ball(v_noise_n, BigFloat(0), BigFloat(0));

In [140]:
@time epsilon_noise = abs(Phi_noise(B0_noise) - B0_noise)

1328.978965 seconds (17.95 G allocations: 1.997 TiB, 21.79% gc time)


Interval(0.0, 1.350262238761869524136625436042572188580093114831929028389356151180103542728806044775961432696441584876592168730302939227706881331159333220013217336806159007377916322721774112772978043645159942621852119283784872371702650887765371661924519744259036029689101211686834993321887271029535308821792430781224832598364560445377837766086559283563011697357702122464878943505703319167734294460860043912624963259871310827139311419732291095e-324)

In [141]:
rho_noise = Interval(BigFloat(10)^Int(round(0.5+log10(epsilon_noise.hi))))

Interval(9.999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999995e-324, 9.999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999995e-324)

In [142]:
B1_noise = Ball(v_noise_n, BigFloat(0), BigFloat(rho_noise.hi));

In [143]:
println(phi(B1_noise).lo)
println(phi(B1_noise).hi)
println(1+rho_noise.hi)

8.24391085425258681839846236502923761606731776662405840926219256825653663924142562689964204720757842422300873689832234963510717328253743947119166688892324018278114543435570594770800377985238316683467659857290704875987642458476648182567707405595689842976849327088118449196788121462757670908015117705258032330416062789993350216685116946928399230707612137155745183941228906543385756780874145679679561145747302048405919208277574503
8.243910854252586818398462365029237616067317766624058409262192568256536639241425626899642047207578424223008736898322349635107173282537439471191666888923240182781145434355705947708003779852383166834676598572907048759876424584766481825677074055956898429768493270881184491967881214627576709080151177052580323304160627899933502186851169469283992307076121371557451839412289065433857567808741456796795611457473020484059192082775745062
1.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [144]:
open(f -> serialize(f, B0_noise), "$(prefix)B0_noise.jls", "w")
open(f -> serialize(f, B1_noise), "$(prefix)B1_noise.jls", "w")

@everywhere begin
B0_noise = deserialize("$(prefix)B0_noise.jls");
B1_noise = deserialize("$(prefix)B1_noise.jls");
end

In [145]:
@everywhere begin
    
function DPhi_noise(b::Ball, L_pp::Array{Interval, 2})
    DF_noise_b = DF_noise(b)
    function DPhi_noise_b(db::Ball)
        DF = DF_noise_b(db)
        DPhi_b_db = db - L_pp(DF, -Interval(1)/phi(V0)^2)
    end
    DPhi_noise_b
end

function bound_kappa_noise_naively(k::Int)
    b = B1_noise
    L_pp = Lambda_PP_noise
    DPhi_noise_b = DPhi_noise(b, L_pp)
    DPhi_noise_b_es_k = DPhi_noise_b(E[k])
    norm = abs(DPhi_noise_b_es_k)
end
    
end

In [146]:
EH = basis_element(B1_noise, N+1)
DT_noise_B1_EH = DT_noise(B1_noise)(EH)
DPhi_noise_B1_EH = (Interval(1)-(phi(B1_noise)/phi(B0_noise))^2)*EH -L_pp(DT_noise_B1_EH, -Interval(1)/phi(V0))
norm_H = abs(DPhi_noise_B1_EH)

Interval(0.0, 4.40687864603670816601150187736421337357698416650754404375682997458988614971479369468394312575819455601361758995345165392909973044157600316778127229757216077048129037637277787868001773045996628559201902684375167704124969249818535600321121865379184118950644144694360823232877430445517139617952878719381981436954798732594389000748079101682276509028371137379614455908790115511949682366078218267811743558289254917958664193431474309e-60)

In [151]:
@everywhere V0 = open(deserialize, "$(prefix)V0.jls")

# note: earlier, when we deserialize B1_noise,
# this ball is centered on V0, we should therefore
# deserialize V0 everywhere, along with it.

In [152]:
workers()

4-element Array{Int64,1}:
  7
  8
  9
 10

In [153]:
@time norms_w = pmap(bound_kappa_noise_naively, [k for k in 1:N+2])

220881.604869 seconds (6.91 M allocations: 164.169 MiB, 0.00% gc time)


642-element Array{Interval,1}:
 Interval(0.0, 1.568900127908808369483342227023643881467810033917360932893030531270683226440722224059353111482186532129216227424136790886360058836777679543116342010618436693669055282613676377176118470055431955246998083028711549135934349564801464489682888089302155052718923871207491136942716355162587837077850597791522084860016113931310999555028389956833915225906563891583941739222389615352650818246104647115539379762425958805283100745093958317e-321)
 Interval(0.0, 4.391237047683788677364753745279886350182116380730027800944127219036101317116529576017872861044539163714897476035044373391594288279153035000692411175618186106073087534253904791207826236099327153620608202487831749259566130016063527734870742023556441992499285438658421197310725111318175210391158929543952261405830184858068704725541427874744147952692054770115313853774541777650727540076929780013250513502804748203369114408106723816e-324)
 Interval(0.0, 4.3890357926029448490619522099099506979407947781323105

In [154]:
norms_w[end] = norm_H
kappa_noise = Interval(hull(norms_w).hi)

Interval(5.861633245587054652783098085434825496988978189615562088927937471400509588171268620451517475244622092813244536172461569170062419963848363011132991013617746107624292054732333949221760295921305543546231734741637554386749589555463080029726342690205494895752746805361960663965401409106719965201691431222761745305235300954230075597391721362059820035041265529816014664777794871233811070147065005539048758607817719756983534591878180552e-49, 5.861633245587054652783098085434825496988978189615562088927937471400509588171268620451517475244622092813244536172461569170062419963848363011132991013617746107624292054732333949221760295921305543546231734741637554386749589555463080029726342690205494895752746805361960663965401409106719965201691431222761745305235300954230075597391721362059820035041265529816014664777794871233811070147065005539048758607817719756983534591878180552e-49)

In [155]:
noise_existence = epsilon_noise < rho_noise*(Interval(1)-kappa_noise)

true

In [157]:
open(f -> serialize(f, norms_w), "$(prefix)norms_w.jls", "w")

In [158]:
open(f -> serialize(f, norms_v), "$(prefix)norms_v.jls", "w")

In [159]:
open(f -> serialize(f, norms), "$(prefix)norms_g.jls", "w")

In [156]:
println("Degree of map at critical point, d:\n$(d)")
println("Domain centre, c:\n$(c)")
println("Domain radius, r:\n$(r)")
println("Truncation degree, N:\n$(N)")
println("Precision (digits approx.), P:\n$(P)")

println("\nepsilon:\n$(epsilon.hi)")
println("rho:\n$(rho)")
println("kappa:\n$(kappa)")

println("\nepsilon_hat:\n$(epsilon_hat.hi)")
println("rho_hat:\n$(rho_hat)")
println("kappa_hat:\n$(kappa_hat.hi)")

println("\nepsilon_noise:\n$(epsilon_noise.hi)")
println("rho_noise:\n$(rho_noise.hi)")
println("kappa_noise:\n$(kappa_noise.hi)")

println("\na=G(1) lower bound:\n$(B1(1).lo)")
println("a=G(1) upper bound:\n$(B1(1).hi)")
println("1+rho:\n$(1+rho)")

println("\nalpha=1/G(1) lower bound:\n$((1/B1(1)).lo)")
println("alpha=1/G(1) upper bound:\n$((1/B1(1)).hi)")
println("1+rho:\n$(1+rho)")

println("\ndelta=phi(V) lower bound:\n$(phi(V1).lo)")
println("delta=phi(V) upper bound:\n$(phi(V1).hi)")
println("1+rho_hat:\n$(1+rho_hat)")

println("\ngamma=phi(K) lower bound:\n$(phi(B1_noise).lo)")
println("gamma=phi(K) upper bound:\n$(phi(B1_noise).hi)")
println("1+rho_noise:\n$(1+rho_noise.hi)")

Degree of map at critical point, d:
4
Domain centre, c:
0.5754000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000005
Domain radius, r:
0.7999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999996
Truncation degree, N:
640
Precision (digits approx.), P:
426

epsi

## Paused here. Resume if completes OK.

In [97]:
rmprocs(procs)

Task (done) @0x00007f7b4775c4f0